In [1]:
import pandas as pd 
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from urllib.parse import urlparse
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error 
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

In [3]:
data = pd.DataFrame(housing.data, columns=housing.feature_names)
data["Price"] = housing.target
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [4]:
data.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
5,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25,2.697
6,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992
7,3.1200,52.0,4.797527,1.061824,1157.0,1.788253,37.84,-122.25,2.414
8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26,2.267
9,3.6912,52.0,4.970588,0.990196,1551.0,2.172269,37.84,-122.25,2.611


In [5]:
X = data.drop(columns=["Price"])
y = data["Price"]

In [6]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [7]:
y

0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Name: Price, Length: 20640, dtype: float64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

In [9]:
X_train

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
7092,5.5238,34.0,5.335740,0.906137,956.0,3.451264,33.92,-118.02
17032,4.5075,39.0,5.250986,1.042050,1744.0,2.291721,37.51,-122.27
6950,4.5595,35.0,5.642361,1.048611,1134.0,3.937500,33.99,-118.07
10160,6.5635,10.0,6.982709,1.025937,1213.0,3.495677,33.90,-117.96
16428,3.5000,14.0,5.552846,1.078591,1071.0,2.902439,38.16,-121.32
...,...,...,...,...,...,...,...,...
15611,6.2578,9.0,7.325967,1.085635,1355.0,3.743094,32.67,-116.89
3776,3.6641,25.0,5.080734,1.038532,1201.0,2.203670,34.16,-118.42
6215,4.5594,33.0,5.351548,1.021858,1519.0,2.766849,34.07,-117.91
4695,3.2010,44.0,4.366721,1.071082,1045.0,1.688207,34.07,-118.37


In [10]:
X_test

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
11115,4.5000,23.0,4.835735,1.063401,912.0,2.628242,33.84,-117.87
17110,15.0001,40.0,8.584541,1.000000,577.0,2.787440,37.46,-122.21
14847,2.9318,26.0,5.406690,1.082746,2156.0,3.795775,32.68,-117.08
7263,2.1912,37.0,2.423077,1.128205,714.0,4.576923,33.99,-118.23
19411,5.7476,19.0,8.045356,1.172786,1412.0,3.049676,37.73,-120.97
...,...,...,...,...,...,...,...,...
9476,2.1625,18.0,5.488136,1.172881,904.0,3.064407,39.42,-123.35
16140,3.2396,52.0,4.021569,1.082353,1400.0,2.745098,37.78,-122.46
1187,2.6071,26.0,5.606695,1.066946,662.0,2.769874,39.44,-121.53
17543,2.8167,52.0,4.145985,1.017032,1336.0,3.250608,37.35,-121.88


In [11]:
y_train

7092     1.853
17032    3.649
6950     1.649
10160    3.469
16428    2.408
         ...  
15611    2.931
3776     3.861
6215     2.042
4695     2.875
9651     1.500
Name: Price, Length: 16512, dtype: float64

In [12]:
y_test

11115    2.37300
17110    5.00001
14847    1.12400
7263     1.12500
19411    2.48600
          ...   
9476     0.77200
16140    3.75000
1187     0.57100
17543    1.83500
11441    2.19700
Name: Price, Length: 4128, dtype: float64

In [13]:
signature = infer_signature(X_train, y_train)

In [14]:
mlflow.set_experiment("House Price Prediction MLFlow Tracking")

2025/07/08 18:17:51 INFO mlflow.tracking.fluent: Experiment with name 'House Price Prediction MLFlow Tracking' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///h:/Data%20Scientists%20Projects/House%20Price%20Prediction/mlruns/652738757862778143', creation_time=1751987871807, experiment_id='652738757862778143', last_update_time=1751987871807, lifecycle_stage='active', name='House Price Prediction MLFlow Tracking', tags={}>

In [15]:
def hyperparameter_tuning(X_train, y_train, param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring="neg_mean_squared_error")
    grid_search.fit(X_train, y_train)
    return grid_search

In [16]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

In [17]:
with mlflow.start_run():
    grid_search = hyperparameter_tuning(X_train, y_train, param_grid)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    mlflow.log_param("best_n_estimators", grid_search.best_params_["n_estimators"])
    mlflow.log_param("best_max_depth", grid_search.best_params_["max_depth"])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_["min_samples_split"])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_["min_samples_leaf"])
    mlflow.log_metric("mse", mse)

    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_uri_type_store != "file":
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="Best RandomForest Model")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)

    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/07/08 18:20:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best RandomForest Model'.
2025/07/08 18:20:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best RandomForest Model, version 1
Created version '1' of model 'Best RandomForest Model'.
2025/07/08 18:20:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run polite-wolf-343 at: http://127.0.0.1:5000/#/experiments/652738757862778143/runs/ce7cc78f45bb4406b0862ab463230735.
2025/07/08 18:20:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/652738757862778143.


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Mean Squared Error: 0.2335835057141934
